In [2]:
import json
import pandas as pd
import os

In [6]:
metadata=pd.read_csv('./resized_test_metadata.csv')
labels=pd.read_csv('./test_labels.csv')


data = {}
data['info'] = {"year": 2021, "version": "1.0", "description": "For object detection", "date_created": "2021"}

data['images'] = []
data['licenses']=[{"id": 1, "name": "GNU General Public License v3.0", "url": "https://github.com/zhiqwang/yolov5-rt-stack/blob/master/LICENSE"}]
data['type']='instances'
data['annotations']=[]
data['categories']=[]
max_imag_id = 0
for ind in range(metadata.shape[0]):
    img_json = {}
    img=metadata.loc[ind]
    img_json['date_captured']='2021'
    img_json['file_name']=img['image_id']+'.bmp'
    id = int(img['image_id'][-4:])
    img_json['image_id']=id-799
#     img_json['image_id']=ind+1
    max_imag_id = ind+1
    img_json['id']=ind+1
    img_json['height']=int(img['height'])
    img_json['width']=int(img['width'])
    data['images'].append(img_json)
max_imag_id += 1

# {"segmentation": 
#     [[1.0799999999999272, 187.69008000000002, 612.66976, 187.69008000000002, 612.66976, 473.53008000000005, 1.0799999999999272, 473.53008000000005]], 
# "area": 174816.81699840003, 
# "iscrowd": 0, 
# "image_id": 1, 
# "bbox": [1.0799999999999272, 187.69008000000002, 611.5897600000001, 285.84000000000003], 
# "category_id": 19, 
# "id": 1}
max_annotation=0
for ind in range(labels.shape[0]):
    anno=labels.loc[ind]
    annotation={}
    annotation['segmentation']=[[]]
    
    annotation['iscrowd']=0
    id = int(anno['image_id'][-4:])
    annotation['image_id']=id-799
#     annotation['image_id']=id+1
    pic = metadata[metadata.image_id==anno['image_id']]
    w=pic['width'].iloc[0]
    h=pic['height'].iloc[0]
    annotation['bbox']=[anno['xmin']*w, anno['ymin']*h, (anno['xmax']-anno['xmin'])*w, (anno['ymax']-anno['ymin'])*h]
    annotation['area']=((anno['ymax']-anno['ymin'])*h)*((anno['xmax']-anno['xmin'])*w)
    if anno['scc']==1:
        annotation['category_id']=1
    elif anno['ac']==1:
        annotation['category_id']=2
    elif anno['sclc']==1:
        annotation['category_id']=3
    elif anno['nsclc']==1 and anno['scc']==0 and anno['ac']==0:
        annotation['category_id']=4
    
    annotation['id']=ind+1
    max_annotation = ind+1
    data['annotations'].append(annotation)

max_annotation+=1



data['categories']=[{"id": 1, "name": "0", "supercategory": "3"},
    {"id": 2, "name": "1", "supercategory": "3"},
    {"id": 3, "name": "2", "supercategory": "0"},
    {"id": 4, "name": "3", "supercategory": "0"}]

print("original finished!!!")   

files = os.listdir('./HUAWEI_MindSpore/dataset_1/merge_test')

n = 0
for file in files:
    names = file.split('_')
    first = 'sample_'+names[-2]
    second = 'sample_'+names[-1][:-4]
    first_pic = metadata.loc[metadata.image_id==first]
    second_pic = metadata.loc[metadata.image_id==second]
    
    img_json = {}
    img_json['date_captured']='2021'
    img_json['file_name']=file
    id = int(img['image_id'][-4:])
#     img_json['image_id']=id-799
    img_json['image_id']=max_imag_id

    img_json['id']=max_imag_id
    w = int(first_pic['width'].iloc[0])+int(second_pic['width'].iloc[0])
    h = max(int(first_pic['height'].iloc[0]),int(second_pic['height'].iloc[0]))
    img_json['width']=w
    img_json['height']=h
    data['images'].append(img_json)
    
    labeles = labels[labels.image_id==first]
    print(labeles)
    for ind in range(len(labeles)):
        anno = labeles.iloc[ind]
        print(anno)
        annotation={}
        annotation['segmentation']=[[]]

        annotation['iscrowd']=0
        id = int(anno['image_id'][-4:])
        annotation['image_id']=max_imag_id
        w=first_pic['width'].iloc[0]
        h=first_pic['height'].iloc[0]
        annotation['bbox']=[anno['xmin']*w, anno['ymin']*h, (anno['xmax']-anno['xmin'])*w, (anno['ymax']-anno['ymin'])*h]
        annotation['area']=((anno['ymax']-anno['ymin'])*h)*((anno['xmax']-anno['xmin'])*w)
        if anno['scc']==1:
            annotation['category_id']=1
        elif anno['ac']==1:
            annotation['category_id']=2
        elif anno['sclc']==1:
            annotation['category_id']=3
        elif anno['nsclc']==1 and anno['scc']==0 and anno['ac']==0:
            annotation['category_id']=4

        annotation['id']=max_annotation
        max_annotation += 1
        print(annotation['bbox'])
        data['annotations'].append(annotation)
    print("first pic finished!!!!!")
    labeles = labels[labels.image_id==second]
    print(labeles)
    for ind in range(len(labeles)):
        
        anno = labeles.iloc[ind]
        print('===',anno)
        annotation={}
        annotation['segmentation']=[[]]

        annotation['iscrowd']=0
        id = int(anno['image_id'][-4:])
        annotation['image_id']=max_imag_id
    #     annotation['image_id']=id+1
        w=second_pic['width'].iloc[0]
        h=second_pic['height'].iloc[0]
        annotation['bbox']=[anno['xmin']*w+first_pic['width'].iloc[0], anno['ymin']*h, (anno['xmax']-anno['xmin'])*w, (anno['ymax']-anno['ymin'])*h]
        annotation['area']=((anno['ymax']-anno['ymin'])*h)*((anno['xmax']-anno['xmin'])*w)
        if anno['scc']==1:
            annotation['category_id']=1
        elif anno['ac']==1:
            annotation['category_id']=2
        elif anno['sclc']==1:
            annotation['category_id']=3
        elif anno['nsclc']==1 and anno['scc']==0 and anno['ac']==0:
            annotation['category_id']=4

        annotation['id']=max_annotation
        max_annotation += 1
        
        print(annotation['bbox'])
        data['annotations'].append(annotation)
    print("second pic finished!!!!")
    
    print(n,',',file,',',max_imag_id)
    n+=1
    max_imag_id += 1
    
with open('resized_test_data.json', 'w') as outfile:
    json.dump(data, outfile)
    

original finished!!!
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
first pic finished!!!!!
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
second pic finished!!!!
0 , merged_image_0801_0873.bmp , 201
       image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
29  sample_0820  0.3266  0.3780  0.5279  0.8539    0   1     0      1
30  sample_0820  0.5979  0.4006  0.7274  0.7063    0   1     0      1
31  sample_0820  0.1218  0.1325  0.2697  0.4864    0   1     0      1
image_id    sample_0820
xmin             0.3266
ymin              0.378
xmax             0.5279
ymax             0.8539
scc                   0
ac                    1
sclc                  0
nsclc                 1
Name: 29, dtype: object
[83.6096, 56.322, 51.53280000000001, 70.9091]
image_id    sample_0820
xmin             0.5979
ymin             0.4006
xmax             0.7274
ymax             0.7063
scc                   0


Name: 280, dtype: object
[446.2336, 50.7196, 24.652800000000013, 17.7161]
=== image_id    sample_0998
xmin             0.4596
ymin             0.4383
xmax             0.5576
ymax             0.5738
scc                   0
ac                    1
sclc                  0
nsclc                 1
Name: 281, dtype: object
[373.6576, 65.3067, 25.087999999999994, 20.189499999999992]
=== image_id    sample_0998
xmin             0.8761
ymin             0.3178
xmax             0.9391
ymax             0.4232
scc                   0
ac                    1
sclc                  0
nsclc                 1
Name: 282, dtype: object
[480.2816, 47.3522, 16.128000000000014, 15.7046]
=== image_id    sample_0998
xmin             0.4115
ymin             0.6702
xmax              0.464
ymax              0.759
scc                   0
ac                    1
sclc                  0
nsclc                 1
Name: 283, dtype: object
[361.344, 99.8598, 13.440000000000012, 13.231199999999998]
=== image_id    sample_

267  sample_0993  0.3295  0.3599  0.4098  0.5105    0   0     0      1
=== image_id    sample_0993
xmin             0.7074
ymin             0.6506
xmax             0.7775
ymax             0.7636
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 266, dtype: object
[437.0944, 87.18039999999999, 17.945599999999985, 15.142]
=== image_id    sample_0993
xmin             0.3295
ymin             0.3599
xmax             0.4098
ymax             0.5105
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 267, dtype: object
[340.352, 48.2266, 20.556799999999996, 20.180399999999995]
second pic finished!!!!
87 , merged_image_0982_0993.bmp , 288
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
first pic finished!!!!!
        image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
266  sample_0993  0.7074  0.6506  0.7775  0.7636    0   0     0      1
26

second pic finished!!!!
119 , merged_image_0908_0947.bmp , 320
        image_id   xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
142  sample_0891  0.028  0.5557  0.1193  0.7063    0   0     1      0
image_id    sample_0891
xmin              0.028
ymin             0.5557
xmax             0.1193
ymax             0.7063
scc                   0
ac                    0
sclc                  1
nsclc                 0
Name: 142, dtype: object
[7.168, 74.46379999999999, 23.3728, 20.18040000000001]
first pic finished!!!!!
        image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
151  sample_0900  0.7406  0.2440  0.9407  0.4608    0   0     1      0
152  sample_0900  0.6014  0.7048  0.6973  0.9488    0   0     1      0
153  sample_0900  0.1188  0.1551  0.2088  0.3133    0   0     1      0
154  sample_0900  0.5539  0.0120  0.6581  0.1446    0   0     1      0
155  sample_0900  0.3947  0.2636  0.5489  0.4292    0   0     1      0
=== image_id    sample_0900
xmin             0.7406

Name: 265, dtype: object
[330.624, 84.184, 60.339200000000005, 47.110400000000006]
second pic finished!!!!
168 , merged_image_0912_0992.bmp , 369
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
first pic finished!!!!!
        image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
198  sample_0946  0.3109  0.4337  0.3837  0.5497    0   0     0      1
=== image_id    sample_0946
xmin             0.3109
ymin             0.4337
xmax             0.3837
ymax             0.5497
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 198, dtype: object
[335.5904, 65.4887, 18.636799999999994, 17.516]
second pic finished!!!!
169 , merged_image_0927_0946.bmp , 370
        image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
209  sample_0955  0.3639  0.1581  0.4706  0.3660    0   0     0      1
210  sample_0955  0.3263  0.3328  0.4781  0.6491    0   0     0      1
211  sample_0955  0.3522  0

In [14]:
files = os.listdir('./HUAWEI_MindSpore/dataset_1/merge_train')
metadata=pd.read_csv('./HUAWEI_MindSpore/dataset_1/resized_train_metadata.csv')
labels=pd.read_csv('./HUAWEI_MindSpore/dataset_1/train_labels.csv')
for file in files:
    names = file.split('_')
    first = 'sample_'+names[-2]
    second = 'sample_'+names[-1][:-4]
    labeles = labels[labels.image_id==first]
    print(file)
print(len(files))

merged_image_0716_0747.bmp
merged_image_0574_0598.bmp
merged_image_0661_0676.bmp
merged_image_0717_0790.bmp
merged_image_0723_0751.bmp
merged_image_0692_0711.bmp
merged_image_0625_0638.bmp
merged_image_0620_0632.bmp
merged_image_0726_0773.bmp
merged_image_0784_0784.bmp
merged_image_0577_0722.bmp
merged_image_0458_0670.bmp
merged_image_0239_0495.bmp
merged_image_0237_0715.bmp
merged_image_0331_0654.bmp
merged_image_0469_0532.bmp
merged_image_0309_0432.bmp
merged_image_0158_0477.bmp
merged_image_0374_0746.bmp
merged_image_0718_0799.bmp
merged_image_0111_0503.bmp
merged_image_0570_0633.bmp
merged_image_0395_0771.bmp
merged_image_0215_0513.bmp
merged_image_0434_0573.bmp
merged_image_0369_0541.bmp
merged_image_0735_0757.bmp
merged_image_0662_0786.bmp
merged_image_0605_0652.bmp
merged_image_0207_0640.bmp
merged_image_0352_0629.bmp
merged_image_0612_0708.bmp
merged_image_0463_0685.bmp
merged_image_0655_0701.bmp
merged_image_0521_0563.bmp
merged_image_0073_0089.bmp
merged_image_0653_0764.bmp
m

merged_image_0329_0641.bmp
merged_image_0580_0696.bmp
merged_image_0016_0262.bmp
merged_image_0642_0708.bmp
merged_image_0015_0594.bmp
merged_image_0371_0636.bmp
merged_image_0445_0486.bmp
merged_image_0779_0792.bmp
merged_image_0765_0797.bmp
merged_image_0676_0734.bmp
merged_image_0529_0700.bmp
merged_image_0491_0670.bmp
merged_image_0003_0180.bmp
merged_image_0054_0505.bmp
merged_image_0131_0766.bmp
merged_image_0623_0651.bmp
merged_image_0122_0420.bmp
merged_image_0380_0502.bmp
merged_image_0569_0793.bmp
merged_image_0013_0242.bmp
merged_image_0501_0511.bmp
merged_image_0568_0787.bmp
merged_image_0601_0655.bmp
merged_image_0075_0208.bmp
merged_image_0233_0303.bmp
merged_image_0494_0726.bmp
merged_image_0038_0438.bmp
merged_image_0557_0745.bmp
merged_image_0657_0788.bmp
merged_image_0700_0771.bmp
merged_image_0275_0691.bmp
merged_image_0489_0509.bmp
merged_image_0746_0750.bmp
merged_image_0411_0640.bmp
merged_image_0260_0798.bmp
merged_image_0660_0727.bmp
merged_image_0278_0580.bmp
m

In [7]:
metadata=pd.read_csv('./resized_train_metadata.csv')
labels=pd.read_csv('./train_labels.csv')


data = {}
data['info'] = {"year": 2021, "version": "1.0", "description": "For object detection", "date_created": "2021"}

data['images'] = []
data['licenses']=[{"id": 1, "name": "GNU General Public License v3.0", "url": "https://github.com/zhiqwang/yolov5-rt-stack/blob/master/LICENSE"}]
data['type']='instances'
data['annotations']=[]
data['categories']=[]
max_imag_id = 0
for ind in range(metadata.shape[0]):
    img_json = {}
    img=metadata.loc[ind]
    img_json['date_captured']='2021'
    img_json['file_name']=img['image_id']+'.bmp'
    id = int(img['image_id'][-4:])
#     img_json['image_id']=id-799
    img_json['image_id']=ind+1
    max_imag_id = ind+1
    img_json['id']=ind+1
    img_json['height']=int(img['height'])
    img_json['width']=int(img['width'])
    data['images'].append(img_json)
max_imag_id += 1

# {"segmentation": 
#     [[1.0799999999999272, 187.69008000000002, 612.66976, 187.69008000000002, 612.66976, 473.53008000000005, 1.0799999999999272, 473.53008000000005]], 
# "area": 174816.81699840003, 
# "iscrowd": 0, 
# "image_id": 1, 
# "bbox": [1.0799999999999272, 187.69008000000002, 611.5897600000001, 285.84000000000003], 
# "category_id": 19, 
# "id": 1}
max_annotation=0
for ind in range(labels.shape[0]):
    anno=labels.loc[ind]
    annotation={}
    annotation['segmentation']=[[]]
    
    annotation['iscrowd']=0
    id = int(anno['image_id'][-4:])
#     annotation['image_id']=id-799
    annotation['image_id']=id+1
    pic = metadata[metadata.image_id==anno['image_id']]
    w=pic['width'].iloc[0]
    h=pic['height'].iloc[0]
    annotation['bbox']=[anno['xmin']*w, anno['ymin']*h, (anno['xmax']-anno['xmin'])*w, (anno['ymax']-anno['ymin'])*h]
    annotation['area']=((anno['ymax']-anno['ymin'])*h)*((anno['xmax']-anno['xmin'])*w)
    if anno['scc']==1:
        annotation['category_id']=1
    elif anno['ac']==1:
        annotation['category_id']=2
    elif anno['sclc']==1:
        annotation['category_id']=3
    elif anno['nsclc']==1 and anno['scc']==0 and anno['ac']==0:
        annotation['category_id']=4
    
    annotation['id']=ind+1
    max_annotation = ind+1
    data['annotations'].append(annotation)

max_annotation+=1



data['categories']=[{"id": 1, "name": "0", "supercategory": "3"},
    {"id": 2, "name": "1", "supercategory": "3"},
    {"id": 3, "name": "2", "supercategory": "0"},
    {"id": 4, "name": "3", "supercategory": "0"}]

print("original finished!!!")   

files = os.listdir('./HUAWEI_MindSpore/dataset_1/merge_train')

n = 0
for file in files:
    names = file.split('_')
    first = 'sample_'+names[-2]
    second = 'sample_'+names[-1][:-4]
    first_pic = metadata.loc[metadata.image_id==first]
    second_pic = metadata.loc[metadata.image_id==second]
    
    img_json = {}
    img_json['date_captured']='2021'
    img_json['file_name']=file
    id = int(img['image_id'][-4:])
#     img_json['image_id']=id-799
    img_json['image_id']=max_imag_id

    img_json['id']=max_imag_id
    w = int(first_pic['width'].iloc[0])+int(second_pic['width'].iloc[0])
    h = max(int(first_pic['height'].iloc[0]),int(second_pic['height'].iloc[0]))
    img_json['width']=w
    img_json['height']=h
    data['images'].append(img_json)
    
    labeles = labels[labels.image_id==first]
    print(labeles)
    for ind in range(len(labeles)):
        anno = labeles.iloc[ind]
        print(anno)
        annotation={}
        annotation['segmentation']=[[]]

        annotation['iscrowd']=0
        id = int(anno['image_id'][-4:])
        annotation['image_id']=max_imag_id
        w=first_pic['width'].iloc[0]
        h=first_pic['height'].iloc[0]
        annotation['bbox']=[anno['xmin']*w, anno['ymin']*h, (anno['xmax']-anno['xmin'])*w, (anno['ymax']-anno['ymin'])*h]
        annotation['area']=((anno['ymax']-anno['ymin'])*h)*((anno['xmax']-anno['xmin'])*w)
        if anno['scc']==1:
            annotation['category_id']=1
        elif anno['ac']==1:
            annotation['category_id']=2
        elif anno['sclc']==1:
            annotation['category_id']=3
        elif anno['nsclc']==1 and anno['scc']==0 and anno['ac']==0:
            annotation['category_id']=4

        annotation['id']=max_annotation
        max_annotation += 1
        print(annotation['bbox'])
        data['annotations'].append(annotation)
    print("first pic finished!!!!!")
    labeles = labels[labels.image_id==second]
    print(labeles)
    for ind in range(len(labeles)):
        
        anno = labeles.iloc[ind]
        print('===',anno)
        annotation={}
        annotation['segmentation']=[[]]

        annotation['iscrowd']=0
        id = int(anno['image_id'][-4:])
        annotation['image_id']=max_imag_id
    #     annotation['image_id']=id+1
        w=second_pic['width'].iloc[0]
        h=second_pic['height'].iloc[0]
        annotation['bbox']=[anno['xmin']*w+first_pic['width'].iloc[0], anno['ymin']*h, (anno['xmax']-anno['xmin'])*w, (anno['ymax']-anno['ymin'])*h]
        annotation['area']=((anno['ymax']-anno['ymin'])*h)*((anno['xmax']-anno['xmin'])*w)
        if anno['scc']==1:
            annotation['category_id']=1
        elif anno['ac']==1:
            annotation['category_id']=2
        elif anno['sclc']==1:
            annotation['category_id']=3
        elif anno['nsclc']==1 and anno['scc']==0 and anno['ac']==0:
            annotation['category_id']=4

        annotation['id']=max_annotation
        max_annotation += 1
        
        print(annotation['bbox'])
        data['annotations'].append(annotation)
    print("second pic finished!!!!")
    
    print(n,',',file,',',max_imag_id)
    n+=1
    max_imag_id += 1
    
with open('resized_train_data.json', 'w') as outfile:
    json.dump(data, outfile)
    

original finished!!!
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
first pic finished!!!!!
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
second pic finished!!!!
0 , merged_image_0405_0619.bmp , 801
         image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
1023  sample_0627  0.6473  0.6958  0.7598  0.8825    0   0     0      1
1024  sample_0627  0.2630  0.0045  0.3980  0.1566    0   0     0      1
1025  sample_0627  0.0946  0.4066  0.3380  0.9247    0   0     0      1
1026  sample_0627  0.3105  0.2334  0.6164  0.8298    0   0     0      1
image_id    sample_0627
xmin             0.6473
ymin             0.6958
xmax             0.7598
ymax             0.8825
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 1023, dtype: object
[165.7088, 98.8036, 28.80000000000001, 26.5114]
image_id    sample_0627
xmin              0.263
ymin        

522  sample_0308  0.3597  0.3178  0.5548  0.6265    0   0     0      1
image_id    sample_0308
xmin             0.5606
ymin             0.8434
xmax             0.6448
ymax             0.9819
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 520, dtype: object
[143.5136, 119.7628, 21.555200000000013, 19.666999999999994]
image_id    sample_0308
xmin             0.1813
ymin             0.0015
xmax             0.2872
ymax             0.1114
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 521, dtype: object
[46.4128, 0.213, 27.110400000000006, 15.6058]
image_id    sample_0308
xmin             0.3597
ymin             0.3178
xmax             0.5548
ymax             0.6265
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 522, dtype: object
[92.0832, 45.1276, 49.945599999999985, 43.835399999999986]
first pic finished!!!!!
         image_i

1074  sample_0666  0.2385  0.6295  0.3901  0.9096    0   0     0      1
=== image_id    sample_0666
xmin             0.5232
ymin             0.1069
xmax             0.6499
ymax              0.369
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 1072, dtype: object
[389.9392, 14.5384, 32.43520000000001, 35.6456]
=== image_id    sample_0666
xmin             0.2578
ymin             0.2605
xmax             0.4735
ymax             0.5994
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 1073, dtype: object
[321.9968, 35.428000000000004, 55.2192, 46.0904]
=== image_id    sample_0666
xmin             0.2385
ymin             0.6295
xmax             0.3901
ymax             0.9096
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 1074, dtype: object
[317.056, 85.612, 38.8096, 38.0936]
second pic finished!!!!
77 , merged_image_0145_0666.bmp ,

954  sample_0574  0.1855  0.244  0.6756  0.8434    0   0     0      1
=== image_id    sample_0574
xmin             0.1855
ymin              0.244
xmax             0.6756
ymax             0.8434
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 954, dtype: object
[303.488, 34.647999999999996, 125.4656, 85.1148]
second pic finished!!!!
112 , merged_image_0184_0574.bmp , 913
        image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
239  sample_0148  0.5247  0.2530  0.6256  0.5678    0   0     0      1
240  sample_0148  0.0971  0.5964  0.2797  0.9367    0   0     0      1
image_id    sample_0148
xmin             0.5247
ymin              0.253
xmax             0.6256
ymax             0.5678
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 239, dtype: object
[134.3232, 35.926, 25.830399999999997, 44.7016]
image_id    sample_0148
xmin             0.0971
ymin             0.5964

461  sample_0264  0.7490  0.3178  0.8298  0.4172    0   0     1      0
image_id    sample_0264
xmin             0.6765
ymin             0.0572
xmax              0.819
ymax             0.2455
scc                   0
ac                    0
sclc                  1
nsclc                 0
Name: 459, dtype: object
[173.184, 8.1224, 36.47999999999999, 26.738599999999998]
image_id    sample_0264
xmin             0.1196
ymin             0.3102
xmax             0.3955
ymax              0.634
scc                   0
ac                    0
sclc                  1
nsclc                 0
Name: 460, dtype: object
[30.6176, 44.048399999999994, 70.63040000000001, 45.979600000000005]
image_id    sample_0264
xmin              0.749
ymin             0.3178
xmax             0.8298
ymax             0.4172
scc                   0
ac                    0
sclc                  1
nsclc                 0
Name: 461, dtype: object
[191.744, 45.1276, 20.684799999999996, 14.114799999999999]
first pic finished!!!

...

Name: 1184, dtype: object
[389.9136, 61.58539999999999, 46.31039999999999, 27.590600000000006]
=== image_id    sample_0743
xmin             0.2405
ymin             0.2349
xmax             0.2872
ymax             0.3223
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 1185, dtype: object
[317.568, 33.3558, 11.955200000000005, 12.410799999999997]
second pic finished!!!!
647 , merged_image_0731_0743.bmp , 1448
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
first pic finished!!!!!
        image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
774  sample_0473  0.3531  0.7214  0.4224  0.8810    0   0     0      1
775  sample_0473  0.0461  0.8117  0.1232  0.9081    0   0     0      1
776  sample_0473  0.2106  0.2485  0.2760  0.3750    0   0     0      1
777  sample_0473  0.3681  0.0316  0.4980  0.3599    0   0     0      1
=== image_id    sample_0473
xmin             0.3531
ymin         

680 , merged_image_0716_0764.bmp , 1481
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
first pic finished!!!!!
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
second pic finished!!!!
681 , merged_image_0383_0754.bmp , 1482
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
first pic finished!!!!!
         image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
1099  sample_0688  0.3905  0.0030  0.6223  0.3027    0   0     0      1
1100  sample_0688  0.2213  0.2605  0.6439  0.8223    0   0     0      1
1101  sample_0688  0.0579  0.7334  0.1305  0.8870    0   0     0      1
1102  sample_0688  0.5964  0.0015  0.8657  0.7425    0   0     0      1
1103  sample_0688  0.0104  0.0030  0.1605  0.2560    0   0     0      1
=== image_id    sample_0688
xmin             0.3905
ymin              0.003
xmax             0.6223
ymax             0.3027
scc             

856  sample_0511  0.4014  0.5060  0.5814  0.8780    0   0     0      1
=== image_id    sample_0511
xmin             0.7015
ymin             0.4111
xmax              0.904
ymax             0.6446
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 853, dtype: object
[435.584, 58.376200000000004, 51.84, 33.15699999999999]
=== image_id    sample_0511
xmin             0.0663
ymin             0.2726
xmax              0.293
ymax             0.6114
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 854, dtype: object
[272.9728, 38.7092, 58.035199999999996, 48.10960000000001]
=== image_id    sample_0511
xmin             0.4355
ymin              0.128
xmax             0.6398
ymax             0.4744
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 855, dtype: object
[367.488, 18.176000000000002, 52.30080000000001, 49.1888]
=== image_id    sampl

653  sample_0391  0.7641  0.6190  0.8303  0.7304    0   0     0      1
image_id    sample_0391
xmin             0.5893
ymin             0.7003
xmax             0.6586
ymax             0.8464
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 652, dtype: object
[150.8608, 93.84020000000001, 17.74079999999998, 19.5774]
image_id    sample_0391
xmin             0.7641
ymin              0.619
xmax             0.8303
ymax             0.7304
scc                   0
ac                    0
sclc                  0
nsclc                 1
Name: 653, dtype: object
[195.6096, 82.946, 16.94720000000001, 14.927600000000007]
first pic finished!!!!!
Empty DataFrame
Columns: [image_id, xmin, ymin, xmax, ymax, scc, ac, sclc, nsclc]
Index: []
second pic finished!!!!
759 , merged_image_0391_0617.bmp , 1560
        image_id    xmin    ymin    xmax    ymax  scc  ac  sclc  nsclc
949  sample_0568  0.7060  0.1642  0.8744  0.4021    0   1     0      1
950  samp

1226  sample_0766  0.2246  0.8554  0.2876  0.9503    0   1     0      1
=== image_id    sample_0766
xmin             0.4119
ymin             0.4277
xmax             0.4784
ymax             0.5301
scc                   0
ac                    1
sclc                  0
nsclc                 1
Name: 1208, dtype: object
[361.4464, 63.72730000000001, 17.024, 15.257599999999998]
=== image_id    sample_0766
xmin             0.2553
ymin             0.1732
xmax             0.3174
ymax             0.2801
scc                   0
ac                    1
sclc                  0
nsclc                 1
Name: 1209, dtype: object
[321.3568, 25.8068, 15.897599999999997, 15.928100000000004]
=== image_id    sample_0766
xmin             0.2806
ymin             0.7816
xmax              0.327
ymax             0.8795
scc                   0
ac                    1
sclc                  0
nsclc                 1
Name: 1210, dtype: object
[327.8336, 116.4584, 11.8784, 14.587099999999998]
=== image_id    sample